In [1]:
import cv2
import os, sys
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw, ImageFont

from cv2 import CascadeClassifier
from PIL import Image
from IPython.display import display, Image, clear_output

import pandas as pd
import dlib

%pylab inline

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense


Populating the interactive namespace from numpy and matplotlib


#### работа с изображениями
- чтение из файла
- распознавание лица
- обрезка изображения (по контуру)
- изменение размера изображения

In [3]:
# для чтения файлов
def viewImage(image, proba): 
    cv2.namedWindow(proba, cv2.WINDOW_NORMAL)
    cv2.imshow(proba, image)
    cv2.destroyAllWindows()

In [2]:
# чтение файлов изображений из папки
#pictures = os.listdir('foto')
#image = cv2.imread('foto/' + pictures[1])
#img = image.copy()
#height, width = img.shape[:2]
#print(height, width)

In [4]:
pictures = os.listdir('foto')
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

n = 0
k = 0
while k < len(pictures):
    # считываем по порядку все изображения
    image = cv2.imread('foto/' + pictures[k])
    
    # работаем только с изображениями. если другие файлы - пропускаем
    if type(image) == np.ndarray:
        # переводим в оттенки серого
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
   
        # высота и ширина картинки
        height, width = gray.shape[:2]

        # каскады Хаара
        faces = face_detector.detectMultiScale(gray, scaleFactor= 1.3, minNeighbors = 15, minSize=(100, 100))

        # Добавим столбец в ndarray faces для создания массива с номером фото
        column_to_be_added = np.array([i for i in range(1, len(faces) + 1)])
        faces_result = np.column_stack((faces, column_to_be_added))

        # обрезка фотографий. к прямоугольному контуру, определенному по признакам Хаара 
        # добавляем 50 по высоте и пропорционально уваличиваем ширину для получения квадрата
                
        for (x, y, w, h, i) in faces_result:
            h1 = h
            if y + h + 50 < height - y:
                if ((x + w/2 + (h + 50)/2) < width) and ((x + w/2 - (h + 50)/2) > 0):
                    h1 = h + 50
                else:
                    h1 = width - x
            else:
                if ((x + w/2 + (height-y)/2) < width) and ((x + w/2 - (height-y)/2) > 0):
                    h1 = height-y
                else:
                    h1 = width - x
            w1 = int(h1/2)
            
            if  (y + h1 < 0 ) or (x + int(w/2) - w1 < 0):
                print(y + h1, x + int(w/2) - w1, pictures[k])
            else: 
                cropped = gray[y: y + h1, x + int(w/2) - w1: x + int(w/2) + w1]
                cv2.imwrite('foto_crop/''c'+ str(n + k + i) +'.jpg', cropped)
            
            
            # масштабируем изображения к одинаковому размеру. 
            # искажения не будет, т.к. обрезка была по квадрату
            img_resize = cv2.resize(cropped, (128, 128), interpolation = cv2.INTER_AREA)
            cv2.imwrite('foto_resized/''r'+str(n + k + i)+'.jpg', img_resize)                     
                    
    else:        
        print('error  foto/' + pictures[k])
    n += len(faces)
    k += 1

#### Создание дата-сета для обучения нейронной сети

In [5]:
# создаем пустой дата-фрем
df_title = {'foto': [], 'kind': []}
df = pd.DataFrame(df_title)

# работаем с уже обработанными изображениями
pictures = os.listdir('foto_resized')
#face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# для определения ключевых точек
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("utils/shape_predictor_68_face_landmarks.dat")

n = 0
k = 0

while k < len(pictures):
    # считываем каждое изображение в папке
    image = cv2.imread('foto_resized/' + pictures[k])
    img_copy = image.copy()
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray_img)    
    
    for face in faces:
        # пустой список и метка для типа лица
        list_face = []
        #face_type = ''
        face_mark = ''
        
        # определяем ключевые точки лица
        landmarks = predictor(gray_img, face)
        
        # основание переносицы
        x_n = landmarks.part(27).x
        y_n = landmarks.part(27).y
        
        # подбородок
        x_gn = landmarks.part(8).x
        y_gn = landmarks.part(8).y
        
        # подвисочные выступы
        x_zy1 = landmarks.part(1).x
        y_zy1 = landmarks.part(1).y
        
        x_zy2 = landmarks.part(15).x
        y_zy2 = landmarks.part(15).y
    
        # челюсть
        x_go_left = landmarks.part(4).x
        y_go_left = landmarks.part(4).y
    
        x_go_right = landmarks.part(12).x
        y_go_right = landmarks.part(12).y
   
        # определяем наиболее выступающие точки челюсти
        for n in range(5, 11):
            if x_go_left > landmarks.part(n).x:
                x_go_left = landmarks.part(n).x
                y_go_left = landmarks.part(n).y
            
            if x_go_right < landmarks.part(n).x:
                x_go_right = landmarks.part(n).x
                y_go_right = landmarks.part(n).y    

        # анатомическая высота        
        n_gn = round(math.sqrt((x_n - x_gn)**2 + (y_n - y_gn)**2))
        # анатомическая ширина
        zy_zy = round(math.sqrt((x_zy1 - x_zy2)**2 + (y_zy1 - y_zy2)**2))

        # коэффициенты для подсчета углов наклона
        m1 = (x_go_right - x_zy2) 
        n1 = (y_go_right - y_zy2)
        
        m3 = (x_go_left - x_zy1) 
        n3 = (y_go_left - y_zy1) 
        
        m2 = (x_go_left - x_go_right)
        n2 = (y_go_left - y_go_right)
       
        # углы наклона касательных
        k_right = math.acos((m1*m2 + n1*n2)/((math.sqrt(m1*m1 + n1*n1))*(math.sqrt(m2*m2 + n2*n2))))
        angle1 = 90 - round((k_right/math.pi)*180)
        
        k_left = math.acos((m1*m2 + n1*n2)/((math.sqrt(m1*m1 + n1*n1))*(math.sqrt(m2*m2 + n2*n2))))
        angle2 = 90 - round((k_left/math.pi)*180)
            
        # определяем тип лица
        if (angle1 > 37) or (angle2 > 27):
            face_mark = '2'
        elif n_gn + 13 < zy_zy:
            face_mark = '1'
        else:
            face_mark = '0'
        
        # добавляем в список словарь   
        list_face.append({'foto': pictures[k], 'kind': face_mark})
        
        # в датафрейм добавляем данные
        df = df.append(list_face)   
    n += len(faces)
    k += 1

In [97]:
# сохраняем датафрейм
df.to_csv('foto_frame1.csv', index=False)